In [12]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import os
import asyncio
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

In [10]:
load_dotenv(override=True)
MAIL_KEY = os.getenv("MAIL_KEY")

In [ ]:
@function_tool
def send_test_email():
    sender_email = "agasthybot@gmail.com"
    receiver_email = "agasthyanathgs@gmail.com"
    app_password   = MAIL_KEY # Replace this

    # Create message
    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = receiver_email
    message["Subject"] = "Test email"

    body = "This is an important test email"
    message.attach(MIMEText(body, "plain"))

    # Connect to Gmail SMTP
    server = smtplib.SMTP("smtp.gmail.com", 587)
    server.starttls()
    server.login(sender_email, app_password)
    server.sendmail(sender_email, receiver_email, message.as_string())
    server.quit()

    print("Email sent successfully!")

send_test_email()


Email sent successfully!


## Step 1: Agent workflow

In [13]:
instructions1 = "You are a sales agent working for BrokeAI, \
a company that provides a Agentic AI solution for customer support and customer complience. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for BrokeAI, \
a company that provides a Agentic AI solution for customer support and customer complience. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for BrokeAI, \
a company that provides a Agentic AI solution for customer support and customer complience. \
You write concise, to the point cold emails."

In [14]:
Agent1 = Agent(name = "proffetional agent",instructions=instructions1,model="gpt-4o-mini")
Agent2 = Agent(name="engaging agent",instructions=instructions2,model="gpt-4o-mini")
Agent3 = Agent(name="busy agent",instructions=instructions3,model="gpt-4o-mini")

In [ ]:
#testing the runner
result =  Runner.run_streamed(Agent1, "Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Transform Your Customer Support with Agentic AI Solutions

Hi [Recipient's Name],

I hope this message finds you well. I’m [Your Name] from BrokeAI, and I wanted to introduce you to an innovative solution that is transforming customer support across various industries.

As organizations increasingly prioritize customer experience, the need for efficient and effective support solutions has never been greater. Our Agentic AI platform empowers teams to provide exceptional service while streamlining operations. Here are a few ways we can help:

- **24/7 Availability**: Ensure your customers receive support whenever they need it, without the delays associated with traditional channels.
- **Consistency and Compliance**: Maintain compliance effortlessly while delivering uniform responses to common inquiries, reducing the risk of errors.
- **Cost Efficiency**: Our solution reduces operational costs by automating routine interactions, allowing your team to focus on more complex issues.

In [39]:
#setting an agent as sales manager who deside the best mail
sales_manager = Agent(name="sales_manager",model="gpt-4o-mini", instructions= "You pick the best cold sales email from the given options. \
                                                                                    Imagine you are a sales manager and pick the one you are most likely to send to customer's. \
                                                                                    Do not give an explanation; reply with the selected email only.")


In [40]:
message = "Write a cold sales email"
with trace("test"):
    results = await asyncio.gather(
        Runner.run(Agent1, message),
        Runner.run(Agent2, message),
        Runner.run(Agent3, message))
    predictions = [result.final_output for result in results]
    emails      = "Cold sales emails:\n\n" + "\n\nEmail:\n\n".join(predictions)
    best_email  = await Runner.run(sales_manager, emails)

    print(f"Best sales email:\n{best_email.final_output}")
    

Best sales email:
Subject: Transform Your Customer Support with AI-Driven Solutions

Hi [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I’m with BrokeAI, where we specialize in advanced Agentic AI solutions designed to enhance customer support and compliance.

In today’s fast-paced environment, delivering exceptional customer support is more crucial than ever. Our AI-driven platform automates responses, streamlines compliance processes, and ensures that your team can focus on what truly matters—building relationships with your customers.

Here are a few ways our solution can benefit your organization:

- **24/7 Support**: Provide instant responses to customer inquiries, reducing wait times and increasing satisfaction.
- **Compliance Assurance**: Automatically monitor and ensure adherence to regulatory standards, minimizing risk.
- **Scalability**: Easily adapt to your growing customer needs without the overhead of additional human resources.

I’d lo

Now go and check out the trace:

https://platform.openai.com/traces

## Steps 2 and 3: Tools and Agent interactions

Remember all that boilerplate json?

Simply wrap your function with the decorator `@function_tool`

In [ ]:
@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("ed@edwarddonner.com")  # Change to your verified sender
    to_email = To("ed.donner@gmail.com")  # Change to your recipient
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sales email", content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

### This has automatically been converted into a tool, with the boilerplate json created

In [ ]:
# Let's look at it
send_email

### And you can also convert an Agent into a tool

In [ ]:
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email")
tool1

### So now we can gather all the tools together:

A tool for each of our 3 email-writing agents

And a tool for our function to send emails

In [ ]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

## And now it's time for our Sales Manager - our planning agent

In [ ]:
# Improved instructions thanks to student Guillermo F.

instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
 
3. Use the send_email tool to send the best email (and only the best email) to the user.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must send ONE email using the send_email tool — never more than one.
"""


sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=tools, model="gpt-4o-mini")

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales manager"):
    result = await Runner.run(sales_manager, message)

### And one more suggestion to send emails from student Oleksandr on Windows 11:

If you are getting certificate SSL errors, then:  
Run this in a terminal: `uv pip install --upgrade certifi`

Then run this code:
```python
import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()
```

Thank you Oleksandr!

## Remember to check the trace

https://platform.openai.com/traces

And then check your email!!


### Handoffs represent a way an agent can delegate to an agent, passing control to it

Handoffs and Agents-as-tools are similar:

In both cases, an Agent can collaborate with another Agent

With tools, control passes back

With handoffs, control passes across



In [ ]:

subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")


In [ ]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("ed@edwarddonner.com")  # Change to your verified sender
    to_email = To("ed.donner@gmail.com")  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [ ]:
tools = [subject_tool, html_tool, send_html_email]

In [ ]:
tools

In [ ]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")


### Now we have 3 tools and 1 handoff

In [ ]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

In [ ]:
# Improved instructions thanks to student Guillermo F.

sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
 
3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

### Remember to check the trace

https://platform.openai.com/traces

And then check your email!!

## Extra note:

Google has released their Agent Development Kit (ADK). It's not yet got the traction of the other frameworks on this course, but it's getting some attention. It's interesting to note that it looks quite similar to OpenAI Agents SDK. To give you a preview, here's a peak at sample code from ADK:

```
root_agent = Agent(
    name="weather_time_agent",
    model="gemini-2.0-flash",
    description="Agent to answer questions about the time and weather in a city.",
    instruction="You are a helpful agent who can answer user questions about the time and weather in a city.",
    tools=[get_weather, get_current_time]
)
```

Well, that looks familiar!

And a student has contributed a customer care agent in community_contributions that uses ADK.